In [1]:
import re
from open_spiel.python.algorithms import cfr, outcome_sampling_mccfr, expected_game_score, exploitability, get_all_states_with_policy
import pyspiel
from open_spiel.python.algorithms.cfr import CFRSolver
import pandas as pd
import pickle
import cProfile, pstats
import numpy as np
from open_spiel.python.algorithms.get_all_states_with_policy import get_all_info_states_with_policy

In [3]:
game = pyspiel.load_game("python_clock_auction", dict(filename='odd_licenses.json'))
cfr_solver = CFRSolver(game)

p = cfr_solver.average_policy()
print(pd.Series(p.action_probability_array.flatten()).value_counts(dropna=False).to_dict())

for it in range(3):
    print(it+1)
    cfr_solver.evaluate_and_update_policy()
    

    # Round
    for k, v in cfr_solver._info_state_nodes.items():
        keys = v.cumulative_regret.keys()
        for k2 in keys:
            v.cumulative_regret[k2] = np.round(v.cumulative_regret[k2], 3)
        keys = v.cumulative_policy.keys()
        for k2 in keys:
            v.cumulative_policy[k2] = np.round(v.cumulative_policy[k2], 3)
            
    regrets = []
    for k, v in cfr_solver._info_state_nodes.items():
        regrets += list(v.cumulative_policy.values())
    s = pd.Series(regrets).value_counts(dropna=False).to_dict()
    
    print("REGRET")
    print(s)
            
    p = cfr_solver.average_policy()
    print("POLICY")
    print(pd.Series(p.action_probability_array.flatten()).value_counts(dropna=False).to_dict())

Game has 322 info states!
{0.0: 556, 0.25: 280, 0.333: 240, 1.0: 132, 0.5: 80}
1
REGRET
{0.001: 145, 0.002: 102, 0.004: 60, 0.003: 53, 0.016: 45, 0.005: 35, 0.008: 32, 0.007: 31, 0.062: 28, 0.009: 27, 0.021: 25, 0.028: 23, 0.031: 20, 0.083: 17, 0.012: 15, 0.25: 13, 0.125: 10, 0.01: 8, 0.047: 8, 0.042: 6, 0.188: 4, 0.014: 4, 1.0: 4, 0.333: 3, 0.037: 3, 0.111: 3, 0.167: 2, 0.056: 2, 0.019: 2, 0.5: 2}
POLICY
{0.0: 556, 0.25: 280, 0.333: 240, 1.0: 132, 0.5: 80}
2
REGRET
{0.001: 145, 0.002: 90, 0.004: 58, 0.003: 43, 0.016: 28, 0.008: 28, 0.005: 21, 0.007: 16, 0.009: 16, 0.031: 16, 0.062: 14, 0.012: 13, 0.028: 12, 0.021: 11, 0.083: 9, 0.121: 7, 0.25: 7, 0.042: 6, 0.143: 6, 0.047: 5, 0.01: 5, 0.125: 4, 0.056: 3, 0.04: 3, 0.752: 3, 0.229: 3, 0.174: 3, 0.061: 3, 0.017: 3, 0.014: 3, 0.46: 3, 0.141: 3, 0.184: 3, 0.034: 3, 0.089: 3, 0.066: 3, 0.018: 3, 0.06: 3, 0.148: 3, 0.188: 3, 0.134: 2, 0.341: 2, 0.451: 2, 0.044: 2, 0.093: 2, 0.553: 2, 0.023: 2, 0.197: 2, 0.122: 2, 0.069: 2, 0.15: 2, 0.775: 2,

In [ ]:
states = get_all_info_states_with_policy(game)

In [ ]:
states_by_history = {}
for state_str in states:
    history = states[state_str]['state'].history()
    states_by_history[tuple(history)] = states[state_str]['state']

In [ ]:
states_by_history[(0,0, 0)].legal_actions()

In [ ]:
cfr_solver

In [ ]:
p = cfr_solver.average_policy()

In [ ]:
import re

def regex(s):
    return re.sub(r'p\d', r'', s)

In [ ]:
# Same info states minus the player...
set(map(regex, p.states_per_player[0])) == set(map(regex, p.states_per_player[1]))

In [ ]:
len(set(map(regex, p.states_per_player[0])))

In [ ]:
pd.Series(p.action_probability_array.flatten()).value_counts(dropna=False).to_dict()

In [ ]:
p.action_probabilities(p.states_per_player[0][0])

In [ ]:
pd.Series(p.action_probability_array[:, 3]).value_counts(dropna=False)

In [ ]:
# { ((P0 activity, P0 processed, P0 submitted), (same for P1)): }
d = dict(game.lottery_cache.items())

for ((p0, p1),) in d:
    print( ((p1, p0),) )
    
    print( d[((p0, p1),)] )
    print( d[((p1, p0),)] )
    
    print()

In [ ]:
(x, y) = list(d.keys())[0][0]